# Roundtrip example

This notebook contains 2 examples of call tree visualizations using *Roundtrip*, an IPython extension that acts as the go-between for the visualization JavaScript and the Python code in the Jupyter notebook. The first example shows how to load a string literal tree into Roundtrip, interact with the tree, and retrieve query information based on the selection on the interactive tree. The second example shows a string literal multi-rooted tree.

**Note: DO NOT USE "RUN ALL", please run the cells manually. Using the Run All feature of Jupyter will produce a known error, caused by the Jupyter magic `%fetchData` because it cannot find the query results to load into `myQuery`**

### Load Roundtrip

In [1]:
import os, sys
from IPython.display import HTML, display

import hatchet as ht

# This is the relative path from the notebook to Roundtrip files in hatchet/external/roundtrip/
roundtrip_path = '../../../hatchet/external/roundtrip/'

# Add the path so that the notebook can find the Roundtrip extension
module_path = os.path.abspath(os.path.join(roundtrip_path)) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Uncomment this line to widen the cells to handle large trees 
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Load the Roundtrip extension. This only needs to be loaded once.
%load_ext roundtrip

## Example 1: Single-rooted tree

In [2]:
# We define the string-literal version of our call tree and load it into a Hatchet GraphFrame
literal_tree = [
{
    "name": "foo",
    "frame": {"name": "foo", "type": "function"},
    "metrics": {"time (inc)": 130.0, "time": 0.0, "module": "a"},
    "children": [
        {
            "name": "bar",
            "frame": {"name": "bar", "type": "function"},
            "metrics": {"time (inc)": 20.0, "time": 5.0, "module": "b"},
            "children": [
                {
                    "name": "baz",
                    "frame": {"name": "baz", "type": "function"},
                    "metrics": {"time (inc)": 5.0, "time": 5.0, "module": "c"},
                },
                {
                    "name": "grault",
                    "frame": {"name": "grault", "type": "function"},
                    "metrics": {"time (inc)": 10.0, "time": 10.0,  "module": "c"},
                },
            ],
        },
        {
            "name": "qux",
            "frame": {"name": "qux", "type": "function"},
            "metrics": {"time (inc)": 60.0, "time": 0.0, "module": "b"},
            "children": [
                {
                    "name": "quux",
                    "frame": {"name": "quux", "type": "function"},
                    "metrics": {"time (inc)": 60.0, "time": 5.0, "module": "c"},
                    "children": [
                        {
                            "name": "corge",
                            "frame": {"name": "corge", "type": "function"},
                            "metrics": {"time (inc)": 55.0, "time": 10.0, "module": "c"},
                            "children": [
                                {
                                    "name": "bar",
                                    "frame": {"name": "bar", "type": "function"},
                                    "metrics": {
                                        "time (inc)": 20.0,
                                        "time": 5.0,
                                        "module": "d"
                                    },
                                    "children": [
                                        {
                                            "name": "baz",
                                            "frame": {"name": "baz", "type": "function"},
                                            "metrics": {
                                                "time (inc)": 5.0,
                                                "time": 5.0,
                                                "module": "d"
                                            },
                                        },
                                        {
                                            "name": "grault",
                                            "frame": {"name": "grault", "type": "function"},
                                            "metrics": {
                                                "time (inc)": 10.0,
                                                "time": 10.0,
                                                "module": "c"
                                            },
                                        },
                                    ],
                                },
                                {
                                    "name": "grault",
                                    "frame": {"name": "grault", "type": "function"},
                                    "metrics": {
                                        "time (inc)": 10.0,
                                        "time": 10.0,
                                        "module": "c"
                                    },
                                },
                                {
                                    "name": "garply",
                                    "frame": {"name": "garply", "type": "function"},
                                    "metrics": {
                                        "time (inc)": 15.0,
                                        "time": 15.0,
                                        "module": "c"
                                    },
                                },
                            ],
                        }
                    ],
                }
            ],
        },
    ],
}
]

### Load the visualization

Next we load the custom visualization from `roundtripTree.js`. The magic function (`%loadVisualization`) and its parameters are listed below:

- `%loadVisualization`: Roundtrip function to generate the visualization
- `roundtrip_path (string)`: the user-specified path to Roundtrip in the Hatchet directory (path ends with `hatchet/external/roundtrip/`)
- `literal_tree (list)`: the user-defined Python variable that defines the tree string-literal (see previous cell)

After the cell is executed, the tree appears. Clicking on a node will cause its metadata to be displayed at the top of the visualization (by the "Colors" button). Double-clicking on a node will cause the subtree to collapse. 

To select a single node, click on it then execute the next cell (`%fetchData`) to retrieve its data. To select many nodes, click the button "Select nodes" to activate the brush (to turn off the brush, click "Select nodes" again).

In [44]:
%loadVisualization roundtrip_path literal_tree

<IPython.core.display.Javascript object>

### Retrieve the selection
Now we retrieve the data we have selected in the tree. The magic function (`%fetchData`) and its parameters are listed below:
- `%fetchData`: the Roundtrip function to pass the selection from JavaScript to Python
- `myQuery (list)`: the user-specified Python variable name we will use to store our selection

In [30]:
# Execute this cell first to populate myQuery with your selection
%fetchData myQuery

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# Now myQuery is loaded and can be used
# If no nodes are clicked/selected, the default behavior is to return a query that shows all nodes.
print(myQuery)

# Note: %fetchData MUST be executed BEFORE printing myQuery. Otherwise, you will
# see the following error: "NameError: name 'myQuery' is not defined"

[{'name': 'bar'}]


In [4]:
# Load the string-literal tree defined above into a GraphFrame
gf = ht.GraphFrame.from_literal(literal_tree)

# Now filter the graphframe gf by the produced query myQuery.
sgf = gf.filter(myQuery, squash=True)

print("Number of input nodes:", len(gf.graph))
print("Number of output nodes:", len(sgf.graph))

# NOTE: The number of output nodes may be more than selected in the 
# Roundtrip tree. This is a known issue in Hatchet: the queries generated using 
# "Select nodes" default to filtering out the new subtree root, and all of its 
# children, down to the leaf nodes.

print(sgf.tree())

NameError: name 'myQuery' is not defined

## Example 2: Multi-rooted tree

The following example uses `multiple_roots`, a call tree representation that has 2 roots: `foo` and `kap`.

In [42]:
multiple_roots = [
    {
        "name": "foo",
        "frame": {"name": "foo", "type": "function"},
        "metrics": {"time (inc)": 130.0, "time": 0.0},
        "children": [
            {
                "name": "bar",
                "frame": {"name": "bar", "type": "function"},
                "metrics": {"time (inc)": 20.0, "time": 5.0},
                "children": [
                    {
                        "name": "baz",
                        "frame": {"name": "baz", "type": "function"},
                        "metrics": {"time (inc)": 5.0, "time": 5.0},
                    },
                    {
                        "name": "grault",
                        "frame": {"name": "grault", "type": "function"},
                        "metrics": {"time (inc)": 10.0, "time": 10.0},
                    },
                ],
            },
            {
                "name": "qux",
                "frame": {"name": "qux", "type": "function"},
                "metrics": {"time (inc)": 60.0, "time": 0.0},
                "children": [
                    {
                        "name": "quux",
                        "frame": {"name": "quux", "type": "function"},
                        "metrics": {"time (inc)": 60.0, "time": 5.0},
                        "children": [
                            {
                                "name": "corge",
                                "frame": {"name": "corge", "type": "function"},
                                "metrics": {"time (inc)": 55.0, "time": 10.0},
                                "children": [
                                    {
                                        "name": "bar",
                                        "frame": {"name": "bar", "type": "function"},
                                        "metrics": {
                                            "time (inc)": 20.0,
                                            "time": 5.0,
                                        },
                                        "children": [
                                            {
                                                "name": "baz",
                                                "frame": {"name": "baz", "type": "function"},
                                                "metrics": {
                                                    "time (inc)": 5.0,
                                                    "time": 5.0,
                                                },
                                            },
                                            {
                                                "name": "grault",
                                                "frame": {"name": "grault", "type": "function"},
                                                "metrics": {
                                                    "time (inc)": 10.0,
                                                    "time": 10.0,
                                                },
                                            },
                                        ],
                                    },
                                    {
                                        "name": "grault",
                                        "frame": {"name": "grault", "type": "function"},
                                        "metrics": {
                                            "time (inc)": 10.0,
                                            "time": 10.0,
                                        },
                                    },
                                    {
                                        "name": "garply", 
                                        "frame": {"name": "garply", "type": "function"},
                                        "metrics": {
                                            "time (inc)": 15.0,
                                            "time": 15.0,
                                        },
                                    },
                                ],
                            }
                        ],
                    }
                ],
            },
            {
                "name": "waldo", 
                "frame": {"name": "waldo", "type": "function"},
                "metrics": {"time (inc)": 50.0, "time": 0.0},
                "children": [
                    {
                        "name": "fred",
                        "frame": {"name": "fred", "type": "function"},
                        "metrics": {"time (inc)": 35.0, "time": 5.0},
                        "children": [
                            {
                                "name": "plugh",
                                "frame": {"name": "plugh", "type": "function"},
                                "metrics": {"time (inc)": 5.0, "time": 5.0},
                            },
                            {
                                "name": "xyzzy", 
                                "frame": {"name": "xyzzy", "type": "function"},
                                "metrics": {"time (inc)": 25.0, "time": 5.0},
                                "children": [
                                    {
                                        "name": "thud", 
                                        "frame": {"name": "thud", "type": "function"},
                                        "metrics": {
                                            "time (inc)": 25.0,
                                            "time": 5.0,
                                        },
                                        "children": [
                                            {
                                                "name": "baz",
                                                "frame": {"name": "baz", "type": "function"},
                                                "metrics": {
                                                    "time (inc)": 5.0,
                                                    "time": 5.0,
                                                },
                                            },
                                            {
                                                "name": "garply",
                                                "frame": {"name": "garply", "type": "function"},
                                                "metrics": {
                                                    "time (inc)": 15.0,
                                                    "time": 15.0,
                                                },
                                            },
                                        ],
                                    }
                                ],
                            },
                        ],
                    },
                    {
                        "name": "garply",
                        "frame": {"name": "garply", "type": "function"},
                        "metrics": {"time (inc)": 15.0, "time": 15.0},
                    },
                ],
            },
        ],
    },      
    {
        "name": "kap", 
        "frame": {"name": "kap", "type": "function"},
        "metrics": {"time (inc)": 30.0, "time": 0.0},
        "children": [
            {
                "name": "kow",
                "frame": {"name": "kow", "type": "function"},
                "metrics": {"time (inc)": 15.0, "time": 5.0},
                "children": [
                    {
                        "name": "krazy",
                        "frame": {"name": "krazy", "type": "function"},
                        "metrics": {"time (inc)": 5.0, "time": 5.0},
                    },
                    {
                        "name": "kraze", 
                        "frame": {"name": "kraze", "type": "function"},
                        "metrics": {"time (inc)": 5.0, "time": 5.0},
                    },
                ],
            },
            {
                "name": "kat",
                "frame": {"name": "kat", "type": "function"},
                "metrics": {"time (inc)": 15.0, "time": 15.0},
            },
        ],
    }
]

Upon load, the tree visualization will show all trees and will color them with a unified legend. Trees can be recolored by their individual minimum and maximum by toggling the "Legends: unified" button to "Legends: indiv."

In [45]:
%loadVisualization roundtrip_path multiple_roots

<IPython.core.display.Javascript object>

In [ ]:
# Execute this cell first to load anotherQuery
%fetchData anotherQuery

In [ ]:
# Now anotherQuery is loaded and can be used (after %fetchData executes)
# If no nodes are clicked/selected, the default behavior is to return a query
# that shows all nodes.
print(anotherQuery)

# Note: %fetchData MUST be executed BEFORE printing anotherQuery. Otherwise, you will
# see the following error: "NameError: name 'anotherQuery' is not defined"

In [ ]:
# Load the string-literal multi-rooted tree defined above into a GraphFrame
multi_gf = ht.GraphFrame.from_literal(multiple_roots)

# Now filter the graphframe by the produced query anotherQuery
multi_sgf = multi_gf.filter(anotherQuery, squash=True)

print("Number of input nodes:", len(multi_gf.graph))
print("Number of output nodes:", len(multi_sgf.graph))

# NOTE: The number of output nodes may be more than selected in the 
# Roundtrip tree. This is a known issue in Hatchet: the queries generated using 
# "Select nodes" default to filtering out the new subtree root, and all of its 
# children, down to the leaf nodes.

print(multi_sgf.tree())